Installing Packages

In [ ]:
#================================================================
#   File name   : multiwalker_v7_PPO
#   Description : multiwalker_v7 Dec-MA PPO continuous with 2 agents 
#   TensorFlow  : 2.3.1
#================================================================

#================================================================
###### Installing and Importing Packages related to train and test our method
#================================================================
!pip install tensorboardX
!pip install gym[All]
#!pip install pettingzoo[sisl]
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # -1:cpu, 0:first gpu
import random
import gym
import pylab
import numpy as np
import tensorflow as tf
from tensorboardX import SummaryWriter
#from pettingzoo.sisl import multiwalker_v7
#tf.config.experimental_run_functions_eagerly(True) # used for debuging and development
tf.compat.v1.disable_eager_execution() # usually using this for fastest performance
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam, RMSprop, Adagrad, Adadelta
from tensorflow.keras import backend as K
import copy

from threading import Thread, Lock
from multiprocessing import Process, Pipe
import time

gpus = tf.config.experimental.list_physical_devices('GPU')
if len(gpus) > 0:
    print(f'GPUs {gpus}')
    try: tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError: pass
#================================================================
###### Installing and Importing Packages related to MultiWalker-v7 Env
#================================================================
!pip install Box2D
!pip install matplotlib-venn
!apt-get -qq install -y libfluidsynth1
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
!apt-get -qq install -y graphviz && pip install pydot
!pip install cartopy
import libarchive
import pydot
import cartopy
from gym import spaces, error
from gym.wrappers.monitoring.video_recorder import ImageEncoder
#import numpy as np
#import time
import scipy

#import copy
import math
import sys
#import gym
#import numpy as np
#from gym import spaces
from gym.utils import colorize, seeding
from six.moves import xrange
import Box2D
from Box2D.b2 import (circleShape, contactListener, edgeShape, fixtureDef, polygonShape,
                      revoluteJointDef)
#================================================================
######## Installing Packages to solve render problem with Colab
#================================================================
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
#!pip install gym[atari] > /dev/null 2>&
!pip install gym[All] > /dev/null 2>&
#import gym
from gym.wrappers import Monitor
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay
from gym.wrappers.monitoring.video_recorder import VideoRecorder

     |████████████████████████████████| 124 kB 11.5 MB/s 
     |████████████████████████████████| 448 kB 12.2 MB/s 
     |████████████████████████████████| 120 kB 50.8 MB/s 
     |████████████████████████████████| 205 kB 41.3 MB/s 
  ERROR: Failed building wheel for mujoco-py
  Running setup.py clean for mujoco-py
Failed to build mujoco-py
    Running setup.py install for mujoco-py ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-kywh0gju/mujoco-py_efd8758b4c7d45b2ab2f2387e3fa152b/setup.py'"'"'; __file__='"'"'/tmp/pip-install-kywh0gju/mujoco-py_efd8758b4c7d45b2ab2f2387e3fa152b/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-

Our MultiWalker-v7 Env from scratch based on 

https://github.com/sisl/MADRL



In [ ]:
def stack_dict_list(dict_list):
    ret = dict()
    if not dict_list:
        return ret
    keys = dict_list[0].keys()
    for k in keys:
        eg = dict_list[0][k]
        if isinstance(eg, dict):
            v = stack_dict_list([x[k] for x in dict_list])
        else:
            v = np.array([x[k] for x in dict_list])
        ret[k] = v

    return ret

class EzPickle(object):
    def __init__(self, *args, **kwargs):
        self._ezpickle_args = args
        self._ezpickle_kwargs = kwargs

    def __getstate__(self):
        return {"_ezpickle_args": self._ezpickle_args, "_ezpickle_kwargs": self._ezpickle_kwargs}

    def __setstate__(self, d):
        out = type(self)(*d["_ezpickle_args"], **d["_ezpickle_kwargs"])
        self.__dict__.update(out.__dict__)
#---------------------------------------------------------------------------------
#---------------------------------------------------------------------------------
class Agent(object):

    def __new__(cls, *args, **kwargs):
        agent = super(Agent, cls).__new__(cls)
        return agent

    @property
    def observation_space(self):
        raise NotImplementedError()

    @property
    def action_space(self):
        raise NotImplementedError()

    def __str__(self):
        return '<{} instance>'.format(type(self).__name__)


class AbstractMAEnv(object):

    def __new__(cls, *args, **kwargs):
        # Easier to override __init__
        env = super(AbstractMAEnv, cls).__new__(cls)
        env._unwrapped = None
        return env

    def setup(self):
        pass

    def seed(self, seed=None):
        return []

    @property
    def agents(self):
        """Returns the agents in the environment. List of objects inherited from Agent class
        Should give us information about cooperating and competing agents?
        """
        raise NotImplementedError()

    @property
    def reward_mech(self):
        raise NotImplementedError()

    def reset(self):
        """Resets the game"""
        raise NotImplementedError()

    def step(self, actions):
        raise NotImplementedError()

    @property
    def is_terminal(self):
        raise NotImplementedError()

    def set_param_values(self, lut):
        for k, v in lut.items():
            setattr(self, k, v)
        self.setup()

    def render(self, *args, **kwargs):
        raise NotImplementedError()

    def animate(self, act_fn, nsteps, **kwargs):
        """act_fn could be a list of functions for each agent in the environemnt that we can control"""
        if not isinstance(act_fn, list):
            act_fn = [act_fn for _ in range(len(self.agents))]
        assert len(act_fn) == len(self.agents)
        encoder = None
        vid_loc = kwargs.pop('vid', None)
        obs = self.reset()
        frame = self.render(**kwargs)
        if vid_loc:
            fps = kwargs.pop('fps', 30)
            encoder = ImageEncoder(vid_loc, frame.shape, fps)
            try:
                encoder.capture_frame(frame)
            except error.InvalidFrame as e:
                print('Invalid video frame, {}'.format(e))

        rew = np.zeros((len(self.agents)))
        traj_info_list = []
        for step in range(nsteps):
            a = list(map(lambda afn, o: afn(o), act_fn, obs))
            obs, r, done, info = self.step(a)
            rew += r
            if info:
                traj_info_list.append(info)

            frame = self.render(**kwargs)
            if vid_loc:
                try:
                    encoder.capture_frame(frame)
                except error.InvalidFrame as e:
                    print('Invalid video frame, {}'.format(e))

            if done:
                break

        traj_info = stack_dict_list(traj_info_list)
        return rew, traj_info

    @property
    def unwrapped(self):
        if self._unwrapped is not None:
            return self._unwrapped
        else:
            return self

    def __str__(self):
        return '<{} instance>'.format(type(self).__name__)


class WrappedAgent(Agent):

    def __init__(self, agent, new_observation_space):
        self._unwrapped = agent
        self._new_observation_space = new_observation_space

    @property
    def observation_space(self):
        return self._new_observation_space

    @property
    def action_space(self):
        return self._unwrapped.action_space

    def unwrapped(self):
        if self.unwrapped is not None:
            return self._unwrapped
        else:
            return self


class ObservationBuffer(AbstractMAEnv):

    def __init__(self, env, buffer_size):
        self._unwrapped = env
        self._buffer_size = buffer_size
        assert all([len(agent.observation_space.shape) == 1 for agent in env.agents])  # XXX
        bufshapes = [tuple(agent.observation_space.shape) + (buffer_size,) for agent in env.agents]
        self._buffer = [np.zeros(bufshape) for bufshape in bufshapes]
        self.reward_mech = self._unwrapped.reward_mech

    @property
    def agents(self):
        aglist = []
        for agid, agent in enumerate(self._unwrapped.agents):
            if isinstance(agent.observation_space, spaces.Box):
                newobservation_space = spaces.Box(low=ent.observation_space.low[0],
                                                  high=agent.observation_space.high[0],
                                                  shape=self._buffer[agid].shape)
            # elif isinstance(agent.observation_sapce, spaces.Discrete):
            else:
                raise NotImplementedError()

            aglist.append(WrappedAgent(agent, newobservation_space))

        return aglist

    @property
    def reward_mech(self):
        return self._unwrapped.reward_mech

    def seed(self, seed=None):
        return self._unwrapped.seed(seed)

    def step(self, action):
        obs, rew, done, info = self._unwrapped.step(action)
        for agid, agid_obs in enumerate(obs):
            self._buffer[agid][..., 0:self._buffer_size - 1] = self._buffer[agid][
                ..., 1:self._buffer_size].copy()
            self._buffer[agid][..., -1] = agid_obs

        bufobs = [buf.copy() for buf in self._buffer]
        return bufobs, rew, done, info

    def reset(self):
        obs = self._unwrapped.reset()

        assert isinstance(obs, list)
        for agid, agid_obs in enumerate(obs):
            for i in range(self._buffer_size):
                self._buffer[agid][..., i] = agid_obs

        bufobs = [buf.copy() for buf in self._buffer]
        return bufobs

    def render(self, *args, **kwargs):
        return self._unwrapped.render(*args, **kwargs)

    def animate(self, *args, **kwargs):
        return self._unwrapped.animate(*args, **kwargs)


class StandardizedEnv(AbstractMAEnv, EzPickle):

    def __init__(self, env, scale_reward=1., enable_obsnorm=False, enable_rewnorm=False,
                 obs_alpha=0.001, rew_alpha=0.001, eps=1e-8):
        EzPickle.__init__(self, env, scale_reward, enable_obsnorm, enable_rewnorm, obs_alpha,
                          rew_alpha, eps)
        self._unwrapped = env
        self._scale_reward = scale_reward
        self._enable_obsnorm = enable_obsnorm
        self._enable_rewnorm = enable_rewnorm
        self._obs_alpha = obs_alpha
        self._rew_alpha = rew_alpha
        self._eps = eps
        self._flatobs_shape = [None for _ in env.agents]
        self._obs_mean = [None for _ in env.agents]
        self._obs_var = [None for _ in env.agents]
        self._rew_mean = [None for _ in env.agents]
        self._rew_var = [None for _ in env.agents]

        for agid, agent in enumerate(env.agents):
            if isinstance(agent.observation_space, spaces.Box):
                self._flatobs_shape[agid] = np.prod(agent.observation_space.shape)
            elif isinstance(env.observation_space, spaces.Discrete):
                self._flatobs_shape[agid] = agent.observation_space.n

            self._obs_mean[agid] = np.zeros(self._flatobs_shape[agid])
            self._obs_var[agid] = np.ones(self._flatobs_shape[agid])
            self._rew_mean[agid] = 0.
            self._rew_var[agid] = 1.

    @property
    def reward_mech(self):
        return self._unwrapped.reward_mech

    @property
    def agents(self):
        return self._unwrapped.agents

    def update_obs_estimate(self, observations):
        for agid, obs in enumerate(observations):
            flatobs = np.asarray(obs).flatten()
            self._obs_mean[agid] = (1 - self._obs_alpha
                                   ) * self._obs_mean[agid] + self._obs_alpha * flatobs
            self._obs_var[agid] = (
                1 - self._obs_alpha
            ) * self._obs_var[agid] + self._obs_alpha * np.square(flatobs - self._obs_mean[agid])

    def update_rew_estimate(self, rewards):
        for agid, reward in enumerate(rewards):
            self._rew_mean[agid] = (1 - self._rew_alpha
                                   ) * self._rew_mean[agid] + self._rew_alpha * reward
            self._rew_var[agid] = (
                1 - self._rew_alpha
            ) * self._rew_var[agid] + self._rew_alpha * np.square(reward - self._rew_mean[agid])

    def standardize_obs(self, observation):
        assert isinstance(observation, list)
        self.update_obs_estimate(observation)
        return [(obs - obsmean) / (np.sqrt(obsvar) + self._eps)
                for (obs, obsmean, obsvar) in zip(observation, self._obs_mean, self._obs_var)]

    def standardize_rew(self, reward):
        assert isinstance(reward, (list, np.ndarray))
        self.update_rew_estimate(reward)
        return [
            rew / (np.sqrt(rewvar) + self._eps)
            for (rew, rewmean, rewvar) in zip(reward, self._rew_mean, self._rew_var)
        ]

    def seed(self, seed=None):
        return self._unwrapped.seed(seed)

    def reset(self):
        obs = self._unwrapped.reset()
        if self._enable_obsnorm:
            return self.standardize_obs(obs)
        else:
            return obs

    def step(self, *args):
        nobslist, rewardlist, done, info = self._unwrapped.step(*args)
        if self._enable_obsnorm:
            nobslist = self.standardize_obs(nobslist)
        if self._enable_rewnorm:
            rewardlist = self.standardize_rew(rewardlist)

        rewardlist = [self._scale_reward * rew for rew in rewardlist]
        return nobslist, rewardlist, done, info

    def __getstate__(self):
        d = EzPickle.__getstate__(self)
        d['_obs_mean'] = self._obs_mean
        d['_obs_var'] = self._obs_var
        return d

    def __setstate__(self, d):
        EzPickle.__setstate__(self, d)
        self._obs_mean = d['_obs_mean']
        self._obs_var = d['_obs_var']

    def __str__(self):
        return "Normalized {}".format(self._unwrapped)

    def render(self, *args, **kwargs):
        return self._unwrapped.render(*args, **kwargs)

    def animate(self, *args, **kwargs):
        return self._unwrapped.animate(*args, **kwargs)


class DiagnosticsWrapper(AbstractMAEnv, EzPickle):

    def __init__(self, env, discount=0.99, max_traj_len=500, log_interval=501):
        self._unwrapped = env
        self._discount = discount
        self._max_traj_len = max_traj_len
        self._episode_time = time.time()
        self._last_time = time.time()
        self._local_t = 0
        self._log_interval = log_interval
        self._episode_reward = np.zeros((len(env.agents)))
        self._episode_length = 0
        self._all_rewards = []

    def reset(self):
        obs = self._unwrapped.reset()
        self._episode_length = 0
        self._episode_reward = np.zeros((len(self._unwrapped.agents)))
        self._all_rewards = []
        return obs

    def step(self, *args):
        obslist, rewardlist, done, info = self._unwrapped.step(*args)
        to_log = {}
        if self._episode_length == 0:
            self._episode_time = time.time()

        self._local_t += 1
        if self._local_t % self._log_interval == 0:
            cur_time = time.time()
            elapsed = cur_time - self._last_time
            fps = self._log_interval / elapsed
            to_log['diagnostics/fps'] = fps
            self._last_time = cur_time

        if rewardlist is not None:
            self._episode_reward += np.asarray(rewardlist)
            self._episode_length += 1
            self._all_rewards.append(rewardlist)

        if done or self._episode_length >= self._max_traj_len:
            total_time = time.time() - self._episode_time
            for agid, epr in enumerate(self._episode_reward):
                to_log['global/episode_reward_agent{}'.format(agid)] = epr

            to_log['global/episode_avg_reward'] = np.mean(self._episode_reward)
            arr = np.asarray(self._all_rewards).mean(axis=1)
            to_log['global/episode_disc_return'] = _discount_sum(arr, self._discount)

            to_log['global/episode_length'] = self._episode_length
            to_log['global/episode_time'] = total_time
            self._episode_length = 0
            self._episode_reward = np.zeros_like(self._episode_reward)
            self._all_rewards = []

        return obslist, rewardlist, done, to_log

    def seed(self, seed=None):
        return self._unwrapped.seed(seed)

    def render(self, *args, **kwargs):
        return self._unwrapped.render(*args, **kwargs)

    def animate(self, *args, **kwargs):
        return self._unwrapped.animate(*args, **kwargs)

    @property
    def reward_mech(self):
        return self._unwrapped.reward_mech

    @property
    def agents(self):
        return self._unwrapped.agents

    def set_param_values(self, lut):
        self._unwrapped.set_param_values(lut)


def _discount_sum(x, discount):
    return np.sum(x * (discount**np.arange(len(x))))

#--------------------------------------------------------------------------
#--------------------------------------------------------------------------
MAX_AGENTS = 40

FPS = 50
SCALE = 30.0  # affects how fast-paced the game is, forces should be adjusted as well

MOTORS_TORQUE = 80
SPEED_HIP = 4
SPEED_KNEE = 6
LIDAR_RANGE = 160 / SCALE

INITIAL_RANDOM = 5

HULL_POLY = [(-30, +9), (+6, +9), (+34, +1), (+34, -8), (-30, -8)]
LEG_DOWN = -8 / SCALE
LEG_W, LEG_H = 8 / SCALE, 34 / SCALE

PACKAGE_POLY = [(-120, 5), (120, 5), (120, -5), (-120, -5)]

PACKAGE_LENGTH = 240

VIEWPORT_W = 600
VIEWPORT_H = 400

TERRAIN_STEP = 14 / SCALE
TERRAIN_LENGTH = 200  # in steps
TERRAIN_HEIGHT = VIEWPORT_H / SCALE / 4
TERRAIN_GRASS = 10  # low long are grass spots, in steps
TERRAIN_STARTPAD = 20  # in steps
FRICTION = 2.5

WALKER_SEPERATION = 10  # in steps


class ContactDetector(contactListener):

    def __init__(self, env):
        contactListener.__init__(self)
        self.env = env

    def BeginContact(self, contact):
        # if walkers fall on ground
        for i, walker in enumerate(self.env.walkers):
            if walker.hull == contact.fixtureA.body:
                if self.env.package != contact.fixtureB.body:
                    self.env.fallen_walkers[i] = True
            if walker.hull == contact.fixtureB.body:
                if self.env.package != contact.fixtureA.body:
                    self.env.fallen_walkers[i] = True

        # if package is on the ground
        if self.env.package == contact.fixtureA.body:
            if contact.fixtureB.body not in [w.hull for w in self.env.walkers]:
                self.env.game_over = True
        if self.env.package == contact.fixtureB.body:
            if contact.fixtureA.body not in [w.hull for w in self.env.walkers]:
                self.env.game_over = True

            #    self.env.game_over = True
        for walker in self.env.walkers:
            for leg in [walker.legs[1], walker.legs[3]]:
                if leg in [contact.fixtureA.body, contact.fixtureB.body]:
                    leg.ground_contact = True

    def EndContact(self, contact):
        for walker in self.env.walkers:
            for leg in [walker.legs[1], walker.legs[3]]:
                if leg in [contact.fixtureA.body, contact.fixtureB.body]:
                    leg.ground_contact = False


class BipedalWalker(Agent):

    def __init__(self, world, init_x=TERRAIN_STEP * TERRAIN_STARTPAD / 2,
                 init_y=TERRAIN_HEIGHT + 2 * LEG_H, n_walkers=2, one_hot=False):
        self.world = world
        self._n_walkers = n_walkers
        self.one_hot = one_hot
        self.hull = None
        self.init_x = init_x
        self.init_y = init_y
        self._seed()

    def _destroy(self):
        if not self.hull:
            return
        self.world.DestroyBody(self.hull)
        self.hull = None
        for leg in self.legs:
            self.world.DestroyBody(leg)
        self.legs = []
        self.joints = []

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def _reset(self):
        self._destroy()

        init_x = self.init_x
        init_y = self.init_y
        self.hull = self.world.CreateDynamicBody(
            position=(init_x, init_y),
            fixtures=fixtureDef(
                shape=polygonShape(vertices=[(x / SCALE, y / SCALE) for x, y in HULL_POLY]),
                density=5.0,
                friction=0.1,
                categoryBits=0x002,
                #maskBits=(0x001 & 0x002),  # collide only with ground
                restitution=0.0)  # 0.99 bouncy
        )
        self.hull.color1 = (0.5, 0.4, 0.9)
        self.hull.color2 = (0.3, 0.3, 0.5)
        self.hull.ApplyForceToCenter((self.np_random.uniform(-INITIAL_RANDOM, INITIAL_RANDOM), 0),
                                     True)

        self.legs = []
        self.joints = []
        for i in [-1, +1]:
            leg = self.world.CreateDynamicBody(
                position=(init_x, init_y - LEG_H / 2 - LEG_DOWN),
                angle=(i * 0.05),
                fixtures=fixtureDef(shape=polygonShape(box=(LEG_W / 2, LEG_H / 2)), density=1.0,
                                    restitution=0.0, categoryBits=0x002,
                                    maskBits=0x001)  # collide with ground only
            )
            leg.color1 = (0.6 - i / 10., 0.3 - i / 10., 0.5 - i / 10.)
            leg.color2 = (0.4 - i / 10., 0.2 - i / 10., 0.3 - i / 10.)
            rjd = revoluteJointDef(
                bodyA=self.hull,
                bodyB=leg,
                localAnchorA=(0, LEG_DOWN),
                localAnchorB=(0, LEG_H / 2),
                enableMotor=True,
                enableLimit=True,
                maxMotorTorque=MOTORS_TORQUE,
                motorSpeed=i,
                lowerAngle=-0.8,
                upperAngle=1.1,)
            self.legs.append(leg)
            self.joints.append(self.world.CreateJoint(rjd))

            lower = self.world.CreateDynamicBody(
                position=(init_x, init_y - LEG_H * 3 / 2 - LEG_DOWN), angle=(i * 0.05),
                fixtures=fixtureDef(shape=polygonShape(box=(0.8 * LEG_W / 2, LEG_H / 2)),
                                    density=1.0, restitution=0.0, categoryBits=0x0020,
                                    maskBits=0x001))
            lower.color1 = (0.6 - i / 10., 0.3 - i / 10., 0.5 - i / 10.)
            lower.color2 = (0.4 - i / 10., 0.2 - i / 10., 0.3 - i / 10.)
            rjd = revoluteJointDef(
                bodyA=leg,
                bodyB=lower,
                localAnchorA=(0, -LEG_H / 2),
                localAnchorB=(0, LEG_H / 2),
                enableMotor=True,
                enableLimit=True,
                maxMotorTorque=MOTORS_TORQUE,
                motorSpeed=1,
                lowerAngle=-1.6,
                upperAngle=-0.1,)
            lower.ground_contact = False
            self.legs.append(lower)
            self.joints.append(self.world.CreateJoint(rjd))

        self.drawlist = self.legs + [self.hull]

        class LidarCallback(Box2D.b2.rayCastCallback):

            def ReportFixture(self, fixture, point, normal, fraction):
                if (fixture.filterData.categoryBits & 1) == 0:
                    return 1
                self.p2 = point
                self.fraction = fraction
                return 0

        self.lidar = [LidarCallback() for _ in range(10)]

    def apply_action(self, action):

        self.joints[0].motorSpeed = float(SPEED_HIP * np.sign(action[0]))
        self.joints[0].maxMotorTorque = float(MOTORS_TORQUE * np.clip(np.abs(action[0]), 0, 1))
        self.joints[1].motorSpeed = float(SPEED_KNEE * np.sign(action[1]))
        self.joints[1].maxMotorTorque = float(MOTORS_TORQUE * np.clip(np.abs(action[1]), 0, 1))
        self.joints[2].motorSpeed = float(SPEED_HIP * np.sign(action[2]))
        self.joints[2].maxMotorTorque = float(MOTORS_TORQUE * np.clip(np.abs(action[2]), 0, 1))
        self.joints[3].motorSpeed = float(SPEED_KNEE * np.sign(action[3]))
        self.joints[3].maxMotorTorque = float(MOTORS_TORQUE * np.clip(np.abs(action[3]), 0, 1))

    def get_observation(self):
        pos = self.hull.position
        vel = self.hull.linearVelocity

        for i in range(10):
            self.lidar[i].fraction = 1.0
            self.lidar[i].p1 = pos
            self.lidar[i].p2 = (pos[0] + math.sin(1.5 * i / 10.0) * LIDAR_RANGE,
                                pos[1] - math.cos(1.5 * i / 10.0) * LIDAR_RANGE)
            self.world.RayCast(self.lidar[i], self.lidar[i].p1, self.lidar[i].p2)

        state = [
            self.hull.angle,  # Normal angles up to 0.5 here, but sure more is possible.
            2.0 * self.hull.angularVelocity / FPS,
            0.3 * vel.x * (VIEWPORT_W / SCALE) / FPS,  # Normalized to get -1..1 range
            0.3 * vel.y * (VIEWPORT_H / SCALE) / FPS,
            self.joints[0].
            angle,  # This will give 1.1 on high up, but it's still OK (and there should be spikes on hiting the ground, that's normal too)
            self.joints[0].speed / SPEED_HIP,
            self.joints[1].angle + 1.0,
            self.joints[1].speed / SPEED_KNEE,
            1.0 if self.legs[1].ground_contact else 0.0,
            self.joints[2].angle,
            self.joints[2].speed / SPEED_HIP,
            self.joints[3].angle + 1.0,
            self.joints[3].speed / SPEED_KNEE,
            1.0 if self.legs[3].ground_contact else 0.0
        ]

        state += [l.fraction for l in self.lidar]
        assert len(state) == 24

        return state

    @property
    def observation_space(self):
        # 24 original obs (joints, etc), 2 displacement obs for each neighboring walker, 3 for package, 1 ID
        idx = MAX_AGENTS if self.one_hot else 1  # TODO
        return spaces.Box(low=-np.inf, high=np.inf, shape=(24 + 4 + 3 + idx,))

    @property
    def action_space(self):
        return spaces.Box(low=-1, high=1, shape=(4,))


class MultiWalkerEnv(AbstractMAEnv, EzPickle):

    metadata = {'render.modes': ['human', 'rgb_array'], 'video.frames_per_second': FPS}

    hardcore = False

    def __init__(self, n_walkers=2, position_noise=1e-3, angle_noise=1e-3, reward_mech='local',
                 forward_reward=1.0, fall_reward=-100.0, drop_reward=-100.0, terminate_on_fall=True,
                 one_hot=False):
        EzPickle.__init__(self, n_walkers, position_noise, angle_noise, reward_mech, forward_reward,
                          fall_reward, drop_reward, terminate_on_fall, one_hot)

        self.n_walkers = n_walkers
        self.position_noise = position_noise
        self.angle_noise = angle_noise
        self._reward_mech = reward_mech
        self.forward_reward = forward_reward
        self.fall_reward = fall_reward
        self.drop_reward = drop_reward
        self.terminate_on_fall = terminate_on_fall
        self.one_hot = one_hot
        self.setup()

    def get_param_values(self):
        return self.__dict__

    def setup(self):
        self.seed()
        self.viewer = None

        self.world = Box2D.b2World()
        self.terrain = None

        init_x = TERRAIN_STEP * TERRAIN_STARTPAD / 2
        init_y = TERRAIN_HEIGHT + 2 * LEG_H
        self.start_x = [
            init_x + WALKER_SEPERATION * i * TERRAIN_STEP for i in range(self.n_walkers)
        ]
        self.walkers = [
            BipedalWalker(self.world, init_x=sx, init_y=init_y, one_hot=self.one_hot)
            for sx in self.start_x
        ]

        self.package_scale = self.n_walkers / 1.75
        self.package_length = PACKAGE_LENGTH / SCALE * self.package_scale

        self.total_agents = self.n_walkers

        self.prev_shaping = np.zeros(self.n_walkers)
        self.prev_package_shaping = 0.0

        self.terrain_length = int(TERRAIN_LENGTH * self.n_walkers * 1 / 8.)

        self.reset()

    @property
    def agents(self):
        return self.walkers

    @property
    def reward_mech(self):
        return self._reward_mech

    def seed(self, seed=None):
        self.np_random, seed_ = seeding.np_random(seed)
        return [seed_]

    def _destroy(self):
        if not self.terrain:
            return
        self.world.contactListener = None
        for t in self.terrain:
            self.world.DestroyBody(t)
        self.terrain = []
        self.world.DestroyBody(self.package)
        self.package = None

        for walker in self.walkers:
            walker._destroy()
            
    def close(self):
        if self.viewer is not None:
            self.viewer.close()
            self.viewer = None
    def reset(self):
        self._destroy()
        self.world.contactListener_bug_workaround = ContactDetector(self)
        self.world.contactListener = self.world.contactListener_bug_workaround
        self.game_over = False
        self.fallen_walkers = np.zeros(self.n_walkers, dtype=np.bool)
        self.prev_shaping = np.zeros(self.n_walkers)
        self.prev_package_shaping = 0.0
        self.scroll = 0.0
        self.lidar_render = 0

        W = VIEWPORT_W / SCALE
        H = VIEWPORT_H / SCALE

        self._generate_package()
        self._generate_terrain(self.hardcore)
        self._generate_clouds()

        self.drawlist = copy.copy(self.terrain)

        self.drawlist += [self.package]

        for walker in self.walkers:
            walker._reset()
            self.drawlist += walker.legs
            self.drawlist += [walker.hull]

        return self.step(np.array([0, 0, 0, 0] * self.n_walkers))[0]

    def step(self, actions):
        act_vec = np.reshape(actions, (self.n_walkers, 4))
        assert len(act_vec) == self.n_walkers
        for i in range(self.n_walkers):
            self.walkers[i].apply_action(act_vec[i])

        self.world.Step(1.0 / FPS, 6 * 30, 2 * 30)

        obs = [walker.get_observation() for walker in self.walkers]

        xpos = np.zeros(self.n_walkers)
        obs = []
        done = False
        rewards = np.zeros(self.n_walkers)

        for i in range(self.n_walkers):
            pos = self.walkers[i].hull.position
            x, y = pos.x, pos.y
            xpos[i] = x

            wobs = self.walkers[i].get_observation()
            nobs = []
            for j in [i - 1, i + 1]:
                # if no neighbor (for edge walkers)
                # if j < 0 or j == self.n_walkers:
                if j < 0 or j == self.n_walkers or self.walkers[j].hull is None:
                    nobs.append(0.0)
                    nobs.append(0.0)
                else:
                    xm = (self.walkers[j].hull.position.x - x) / self.package_length
                    ym = (self.walkers[j].hull.position.y - y) / self.package_length
                    nobs.append(np.random.normal(xm, self.position_noise))
                    nobs.append(np.random.normal(ym, self.position_noise))
            xd = (self.package.position.x - x) / self.package_length
            yd = (self.package.position.y - y) / self.package_length
            nobs.append(np.random.normal(xd, self.position_noise))
            nobs.append(np.random.normal(yd, self.position_noise))
            nobs.append(np.random.normal(self.package.angle, self.angle_noise))
            # ID # we cand delete this part to have obs with 31 element 
            if self.one_hot:
                nobs.extend(np.eye(MAX_AGENTS)[i])
            else:
                nobs.append(float(i) / self.n_walkers)
            obs.append(np.array(wobs + nobs))

            #shaping = 130 * pos[0] / SCALE
            shaping = 0.0
            shaping -= 5.0 * abs(wobs[0])
            rewards[i] = shaping - self.prev_shaping[i]
            self.prev_shaping[i] = shaping

        package_shaping = self.forward_reward * 130 * self.package.position.x / SCALE
        rewards += (package_shaping - self.prev_package_shaping)
        self.prev_package_shaping = package_shaping

        self.scroll = xpos.mean() - VIEWPORT_W / SCALE / 5 - (self.n_walkers - 1
                                                             ) * WALKER_SEPERATION * TERRAIN_STEP

        done = False
        if self.game_over or pos[0] < 0:
            rewards += self.drop_reward
            done = True
        if pos[0] > (self.terrain_length - TERRAIN_GRASS) * TERRAIN_STEP:
            done = True
        rewards += self.fall_reward * self.fallen_walkers
        if self.terminate_on_fall and np.sum(self.fallen_walkers) > 0:
            done = True

        if self.reward_mech == 'local':
            return obs, rewards, done, {}
        return obs, [rewards.mean()] * self.n_walkers, done, {}

    def render(self, mode='human', close=False):
        if close:
            if self.viewer is not None:
                self.viewer.close()
                self.viewer = None
            return

        render_scale = 0.75

        from gym.envs.classic_control import rendering
        if self.viewer is None:
            self.viewer = rendering.Viewer(VIEWPORT_W, VIEWPORT_H)
        self.viewer.set_bounds(self.scroll,
                               VIEWPORT_W / SCALE * self.package_scale * render_scale + self.scroll,
                               0, VIEWPORT_H / SCALE * self.package_scale * render_scale)

        self.viewer.draw_polygon([
            (self.scroll, 0),
            (self.scroll + VIEWPORT_W * self.package_scale / SCALE * render_scale, 0),
            (self.scroll + VIEWPORT_W * self.package_scale / SCALE * render_scale,
             VIEWPORT_H / SCALE * self.package_scale * render_scale),
            (self.scroll, VIEWPORT_H / SCALE * self.package_scale * render_scale),
        ], color=(0.9, 0.9, 1.0))
        for poly, x1, x2 in self.cloud_poly:
            if x2 < self.scroll / 2:
                continue
            if x1 > self.scroll / 2 + VIEWPORT_W / SCALE * self.package_scale:
                continue
            self.viewer.draw_polygon([(p[0] + self.scroll / 2, p[1]) for p in poly], color=(1, 1,
                                                                                            1))
        for poly, color in self.terrain_poly:
            if poly[1][0] < self.scroll:
                continue
            if poly[0][0] > self.scroll + VIEWPORT_W / SCALE * self.package_scale:
                continue
            self.viewer.draw_polygon(poly, color=color)

        self.lidar_render = (self.lidar_render + 1) % 100
        i = self.lidar_render
        for walker in self.walkers:
            if i < 2 * len(walker.lidar):
                l = walker.lidar[i] if i < len(walker.lidar) else walker.lidar[len(walker.lidar)
                                                                               - i - 1]
                self.viewer.draw_polyline([l.p1, l.p2], color=(1, 0, 0), linewidth=1)

        for obj in self.drawlist:
            for f in obj.fixtures:
                trans = f.body.transform
                if type(f.shape) is circleShape:
                    t = rendering.Transform(translation=trans * f.shape.pos)
                    self.viewer.draw_circle(f.shape.radius, 30, color=obj.color1).add_attr(t)
                    self.viewer.draw_circle(f.shape.radius, 30, color=obj.color2, filled=False,
                                            linewidth=2).add_attr(t)
                else:
                    path = [trans * v for v in f.shape.vertices]
                    self.viewer.draw_polygon(path, color=obj.color1)
                    path.append(path[0])
                    self.viewer.draw_polyline(path, color=obj.color2, linewidth=2)

        flagy1 = TERRAIN_HEIGHT
        flagy2 = flagy1 + 50 / SCALE
        x = TERRAIN_STEP * 3
        self.viewer.draw_polyline([(x, flagy1), (x, flagy2)], color=(0, 0, 0), linewidth=2)
        f = [(x, flagy2), (x, flagy2 - 10 / SCALE), (x + 25 / SCALE, flagy2 - 5 / SCALE)]
        self.viewer.draw_polygon(f, color=(0.9, 0.2, 0))
        self.viewer.draw_polyline(f + [f[0]], color=(0, 0, 0), linewidth=2)

        return self.viewer.render(return_rgb_array=mode == 'rgb_array')

    def _generate_package(self):
        init_x = np.mean(self.start_x)
        init_y = TERRAIN_HEIGHT + 3 * LEG_H
        self.package = self.world.CreateDynamicBody(
            position=(init_x, init_y),
            fixtures=fixtureDef(
                shape=polygonShape(vertices=[(x * self.package_scale / SCALE, y / SCALE)
                                             for x, y in PACKAGE_POLY]),
                density=1.0,
                friction=0.5,
                categoryBits=0x004,
                #maskBits=0x001,  # collide only with ground
                restitution=0.0)  # 0.99 bouncy
        )
        self.package.color1 = (0.5, 0.4, 0.9)
        self.package.color2 = (0.3, 0.3, 0.5)

    def _generate_terrain(self, hardcore):
        GRASS, STUMP, STAIRS, PIT, _STATES_ = range(5)
        state = GRASS
        velocity = 0.0
        y = TERRAIN_HEIGHT
        counter = TERRAIN_STARTPAD
        oneshot = False
        self.terrain = []
        self.terrain_x = []
        self.terrain_y = []
        for i in range(self.terrain_length):
            x = i * TERRAIN_STEP
            self.terrain_x.append(x)

            if state == GRASS and not oneshot:
                velocity = 0.8 * velocity + 0.01 * np.sign(TERRAIN_HEIGHT - y)
                if i > TERRAIN_STARTPAD:
                    velocity += self.np_random.uniform(-1, 1) / SCALE  #1
                y += velocity

            elif state == PIT and oneshot:
                counter = self.np_random.randint(3, 5)
                poly = [
                    (x, y),
                    (x + TERRAIN_STEP, y),
                    (x + TERRAIN_STEP, y - 4 * TERRAIN_STEP),
                    (x, y - 4 * TERRAIN_STEP),
                ]
                t = self.world.CreateStaticBody(fixtures=fixtureDef(
                    shape=polygonShape(vertices=poly), friction=FRICTION))
                t.color1, t.color2 = (1, 1, 1), (0.6, 0.6, 0.6)
                self.terrain.append(t)
                t = self.world.CreateStaticBody(fixtures=fixtureDef(shape=polygonShape(
                    vertices=[(p[0] + TERRAIN_STEP * counter, p[1]) for p in poly]),
                                                                    friction=FRICTION))
                t.color1, t.color2 = (1, 1, 1), (0.6, 0.6, 0.6)
                self.terrain.append(t)
                counter += 2
                original_y = y

            elif state == PIT and not oneshot:
                y = original_y
                if counter > 1:
                    y -= 4 * TERRAIN_STEP

            elif state == STUMP and oneshot:
                counter = self.np_random.randint(1, 3)
                poly = [
                    (x, y),
                    (x + counter * TERRAIN_STEP, y),
                    (x + counter * TERRAIN_STEP, y + counter * TERRAIN_STEP),
                    (x, y + counter * TERRAIN_STEP),
                ]
                t = self.world.CreateStaticBody(fixtures=fixtureDef(
                    shape=polygonShape(vertices=poly), friction=FRICTION))
                t.color1, t.color2 = (1, 1, 1), (0.6, 0.6, 0.6)
                self.terrain.append(t)

            elif state == STAIRS and oneshot:
                stair_height = +1 if self.np_random.rand() > 0.5 else -1
                stair_width = self.np_random.randint(4, 5)
                stair_steps = self.np_random.randint(3, 5)
                original_y = y
                for s in range(stair_steps):
                    poly = [
                        (x + (s * stair_width) * TERRAIN_STEP,
                         y + (s * stair_height) * TERRAIN_STEP),
                        (x + ((1 + s) * stair_width) * TERRAIN_STEP,
                         y + (s * stair_height) * TERRAIN_STEP),
                        (x + ((1 + s) * stair_width) * TERRAIN_STEP,
                         y + (-1 + s * stair_height) * TERRAIN_STEP),
                        (x + (s * stair_width) * TERRAIN_STEP,
                         y + (-1 + s * stair_height) * TERRAIN_STEP),
                    ]
                    t = self.world.CreateStaticBody(fixtures=fixtureDef(
                        shape=polygonShape(vertices=poly), friction=FRICTION))
                    t.color1, t.color2 = (1, 1, 1), (0.6, 0.6, 0.6)
                    self.terrain.append(t)
                counter = stair_steps * stair_width

            elif state == STAIRS and not oneshot:
                s = stair_steps * stair_width - counter - stair_height
                n = s / stair_width
                y = original_y + (n * stair_height) * TERRAIN_STEP

            oneshot = False
            self.terrain_y.append(y)
            counter -= 1
            if counter == 0:
                counter = self.np_random.randint(TERRAIN_GRASS / 2, TERRAIN_GRASS)
                if state == GRASS and hardcore:
                    state = self.np_random.randint(1, _STATES_)
                    oneshot = True
                else:
                    state = GRASS
                    oneshot = True

        self.terrain_poly = []
        for i in range(self.terrain_length - 1):
            poly = [(self.terrain_x[i], self.terrain_y[i]),
                    (self.terrain_x[i + 1], self.terrain_y[i + 1])]
            t = self.world.CreateStaticBody(fixtures=fixtureDef(
                shape=edgeShape(vertices=poly),
                friction=FRICTION,
                categoryBits=0x0001,))
            color = (0.3, 1.0 if i % 2 == 0 else 0.8, 0.3)
            t.color1 = color
            t.color2 = color
            self.terrain.append(t)
            color = (0.4, 0.6, 0.3)
            poly += [(poly[1][0], 0), (poly[0][0], 0)]
            self.terrain_poly.append((poly, color))
        self.terrain.reverse()

    def _generate_clouds(self):
        # Sorry for the clouds, couldn't resist
        self.cloud_poly = []
        for i in range(self.terrain_length // 20):
            x = self.np_random.uniform(0, self.terrain_length) * TERRAIN_STEP
            y = VIEWPORT_H / SCALE * 3 / 4
            poly = [(x + 15 * TERRAIN_STEP * math.sin(3.14 * 2 * a / 5) + self.np_random.uniform(
                0, 5 * TERRAIN_STEP), y + 5 * TERRAIN_STEP * math.cos(3.14 * 2 * a / 5) +
                     self.np_random.uniform(0, 5 * TERRAIN_STEP)) for a in range(5)]
            x1 = min([p[0] for p in poly])
            x2 = max([p[0] for p in poly])
            self.cloud_poly.append((poly, x1, x2))
#--------------------------------------------------------------------------
#--------------------------------------------------------------------------
#if __name__ == "__main__":
#  n_walkers=2
#  env = MultiWalkerEnv(n_walkers=n_walkers, position_noise=1e-3, angle_noise=1e-3, reward_mech='global',
#                 forward_reward=1.0, fall_reward=-100.0, drop_reward=-10, terminate_on_fall=True,
#                 one_hot=False) #drop_reward=-100
#  env.reset()
#  for i in range(3):
#      env.render()
#      a = np.array([env.agents[0].action_space.sample() for _ in range(n_walkers)])
#      o, r, done, _ = env.step(a)
#      print("\nStep:", i)
#      print ("Obs:", o)
#      print("Rewards:", r)
#      print ("Term:", done)
#      if done:
#          break

Defining funnction to solve render problem with Colab

In [ ]:
display = Display(visible=0, size=(1400, 900))
display.start()

def show_video(t):
  mp4list = glob.glob('*.mp4')
  if len(mp4list) > 0:
    mp4 = t+'.mp4'
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find this video")

Random Action to test import Env and render function

In [ ]:
import random
n_walkers=2
#env=multiwalker_v7.env(n_walkers=2, position_noise=1e-3, angle_noise=1e-3, local_ratio=0, 
#                       forward_reward=1.0, terminate_reward=-100.0, fall_reward=-10.0,
#                       terminate_on_fall=True, remove_on_fall=True, max_cycles=500) #local_ratio: Proportion of reward allocated locally vs distributed among all agents
env = MultiWalkerEnv(n_walkers=n_walkers, position_noise=1e-3, angle_noise=1e-3, reward_mech='global',
                 forward_reward=1.0, fall_reward=-100.0, drop_reward=-10, terminate_on_fall=True,
                 one_hot=False) #drop_reward=-100
video_recorder = None
video_recorder = VideoRecorder(env,'./random.mp4', enabled=True)
env.reset()
def Random_games():
    #action_size = env.action_spaces[env.agents[0]].shape[0]
    action_size = env.agents[0].action_space.shape[0]
    for episode in range(3): #40
        env.reset()
        while True:
            env.render()
            video_recorder.capture_frame() 
            action_env1 = np.random.uniform(-1.0, 1.0, size=action_size)
            action_env2 = np.random.uniform(-1.0, 1.0, size=action_size)
            actions_env = np.array((action_env1, action_env2)) #or
            #actions_env = np.array([env.agents[0].action_space.sample() for _ in range(n_walkers)])
            #print(actions_env)
            next_state, reward, done, info = env.step(actions_env)
            if done:
                break
    print('Saved video.')
    video_recorder.close()
    video_recorder.enabled = False
    env.close()
                    
Random_games()
show_video('random')

Saved video.


Check input and output shapes

In [ ]:
n_walkers=2
env = MultiWalkerEnv(n_walkers=n_walkers, position_noise=1e-3, angle_noise=1e-3, reward_mech='global',
              forward_reward=1.0, fall_reward=-100.0, drop_reward=-10, terminate_on_fall=True,
              one_hot=False) #drop_reward=-100
env.reset()
for i in range(2):
    env.render()
    a = np.array([env.agents[0].action_space.sample() for _ in range(n_walkers)])
    o, r, done, _ = env.step(a)
    print("\nStep:", i)
    print("action:", a)
    print ("Obs:", o)
    print("Rewards:", r)
    print ("Term:", done)
    if done:
        break


Step: 0
action: [[ 0.27676868  0.16777876  0.35842466  0.9011866 ]
 [ 0.13628882  0.36590016  0.6532219  -0.24308984]]
Obs: [array([ 7.06893299e-03, -1.82135510e-02,  7.01421909e-04,  3.14663911e-02,
       -2.53710896e-01, -1.39710471e-01,  1.44060364e+00,  8.17169825e-01,
        1.00000000e+00, -3.61738861e-01, -6.26165450e-01,  1.69497693e+00,
        9.97367859e-01,  1.00000000e+00,  4.57857937e-01,  4.63057607e-01,
        4.79263544e-01,  5.08478463e-01,  5.54753721e-01,  6.25754952e-01,
        7.36567855e-01,  9.20186102e-01,  1.00000000e+00,  1.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  5.08416080e-01, -3.61595442e-03,
        2.55578275e-01,  1.02544800e-01, -1.13245572e-03,  0.00000000e+00]), array([-6.99412497e-03, -2.68854642e-02, -1.14728692e-02,  1.69814146e-02,
       -2.88102031e-01, -4.00148928e-01,  1.78094965e+00,  7.93457349e-01,
        1.00000000e+00,  3.24009776e-01,  9.69762564e-01,  1.44447803e-01,
       -9.99963919e-01,  1.00000000e+00,  4.517

In [ ]:
# class Environment(Process):
#     def __init__(self, env_idx, child_conn, env_name, state_size, action_size, visualize=False):
#         super(Environment, self).__init__()
#         self.env = gym.make(env_name) #!
#         self.is_render = visualize
#         self.env_idx = env_idx
#         self.child_conn = child_conn
#         self.state_size = state_size
#         self.action_size = action_size

#     def run(self):
#         super(Environment, self).run()
#         state = self.env.reset() #!!!
#         state = np.reshape(state, [1, self.state_size]) #!!!
#         self.child_conn.send(state) #!!!
#         while True:
#             action = self.child_conn.recv()
#             #if self.is_render and self.env_idx == 0:
#                 #self.env.render()

#             state, reward, done, info = self.env.step(action)
#             state = np.reshape(state, [1, self.state_size])

#             if done:
#                 state = self.env.reset()
#                 state = np.reshape(state, [1, self.state_size])

#             self.child_conn.send([state, reward, done, info])

In [ ]:
class Actor_Model:
    def __init__(self, input_shape, action_space, lr, optimizer):
        X_input = Input(input_shape)
        self.action_space = action_space
        
        X = Dense(512, activation="relu", kernel_initializer=tf.random_normal_initializer(stddev=0.01))(X_input)
        X = Dense(256, activation="relu", kernel_initializer=tf.random_normal_initializer(stddev=0.01))(X)
        X = Dense(64, activation="relu", kernel_initializer=tf.random_normal_initializer(stddev=0.01))(X)
        output = Dense(self.action_space, activation="tanh")(X)

        self.Actor = Model(inputs = X_input, outputs = output)
        self.Actor.compile(loss=self.ppo_loss_continuous, optimizer=optimizer(lr=lr))
        #print(self.Actor.summary())

    def ppo_loss_continuous(self, y_true, y_pred):
        advantages, actions, logp_old_ph, = y_true[:, :1], y_true[:, 1:1+self.action_space], y_true[:, 1+self.action_space]
        LOSS_CLIPPING = 0.2
        logp = self.gaussian_likelihood(actions, y_pred)

        ratio = K.exp(logp - logp_old_ph)

        p1 = ratio * advantages
        p2 = tf.where(advantages > 0, (1.0 + LOSS_CLIPPING)*advantages, (1.0 - LOSS_CLIPPING)*advantages) # minimum advantage

        actor_loss = -K.mean(K.minimum(p1, p2))

        return actor_loss

    def gaussian_likelihood(self, actions, pred): # for keras custom loss
        log_std = -0.5 * np.ones(self.action_space, dtype=np.float32)
        pre_sum = -0.5 * (((actions-pred)/(K.exp(log_std)+1e-8))**2 + 2*log_std + K.log(2*np.pi))
        return K.sum(pre_sum, axis=1)

    def predict(self, state):
        return self.Actor.predict(state)

In [ ]:
class Critic_Model:
    def __init__(self, input_shape, action_space, lr, optimizer):
        X_input = Input(input_shape)
        old_values = Input(shape=(1,))

        V = Dense(512, activation="relu", kernel_initializer=tf.random_normal_initializer(stddev=0.01))(X_input)
        V = Dense(256, activation="relu", kernel_initializer=tf.random_normal_initializer(stddev=0.01))(V)
        V = Dense(64, activation="relu", kernel_initializer=tf.random_normal_initializer(stddev=0.01))(V)
        value = Dense(1, activation=None)(V)

        self.Critic = Model(inputs=[X_input, old_values], outputs = value)
        self.Critic.compile(loss=[self.critic_PPO2_loss(old_values)], optimizer=optimizer(lr=lr))

    def critic_PPO2_loss(self, values):
        def loss(y_true, y_pred):
            LOSS_CLIPPING = 0.2
            clipped_value_loss = values + K.clip(y_pred - values, -LOSS_CLIPPING, LOSS_CLIPPING)
            v_loss1 = (y_true - clipped_value_loss) ** 2
            v_loss2 = (y_true - y_pred) ** 2
            
            value_loss = 0.5 * K.mean(K.maximum(v_loss1, v_loss2))
            #value_loss = K.mean((y_true - y_pred) ** 2) # standard PPO loss
            return value_loss
        return loss

    def predict(self, state):
        return self.Critic.predict([state, np.zeros((state.shape[0], 1))])

In [ ]:
class PPOAgent:
    # PPO Main Optimization Algorithm
    def __init__(self, env_name, model_name=""):
        # Initialization
        # Environment and PPO parameters
        self.env_name = env_name
        self.env = MultiWalkerEnv(n_walkers=n_walkers, position_noise=1e-3, angle_noise=1e-3, 
                                  reward_mech='global', forward_reward=1.0, fall_reward=-100.0, 
                                  drop_reward=-10, terminate_on_fall=True, one_hot=False) #drop_reward=-100    
        self.action_size = self.env.agents[0].action_space.shape[0] #=4
        self.state_size = self.env.agents[0].observation_space.shape #=(32,)
        self.EPISODES = 200000 #######200000 # total episodes to train through all environments
        self.episode = 0 # used to track the episodes total count of episodes played through all thread environments
        self.max_average_env1 = 0 # when average score is above 0 model will be saved
        self.max_average_env2 = 0
        self.lr = 0.00025
        self.epochs = 10 # training epochs
        self.shuffle = True #?
        self.Training_batch = 512
        #self.optimizer = RMSprop
        self.optimizer = Adam

        self.replay_count_env1 = 0
        self.replay_count_env2 = 0
        self.writer = SummaryWriter(comment="_"+self.env_name+"_"+self.optimizer.__name__+"_"+str(self.lr))
        
        # Instantiate plot memory
        self.scores_, self.episodes_, self.average_ = [], [], [] # used in matplotlib plots

        # Create Actor-Critic network models
        self.Actor_env1 = Actor_Model(input_shape=self.state_size, action_space = self.action_size, lr=self.lr, optimizer = self.optimizer)
        self.Actor_env2 = Actor_Model(input_shape=self.state_size, action_space = self.action_size, lr=self.lr, optimizer = self.optimizer)
        self.Critic_env1 = Critic_Model(input_shape=self.state_size, action_space = self.action_size, lr=self.lr, optimizer = self.optimizer)
        self.Critic_env2 = Critic_Model(input_shape=self.state_size, action_space = self.action_size, lr=self.lr, optimizer = self.optimizer)

        self.Actor_name_env1 = f"{self.env_name}_PPO_Actor_env1.h5"
        self.Actor_name_env2 = f"{self.env_name}_PPO_Actor_env2.h5"
        self.Critic_name_env1 = f"{self.env_name}_PPO_Critic_env1.h5"
        self.Critic_name_env2 = f"{self.env_name}_PPO_Critic_env2.h5"
        #self.load() # uncomment to continue training from old weights #!!!!!!!!!!!!!

        # do not change bellow
        self.log_std = -0.5 * np.ones(self.action_size, dtype=np.float32)
        self.std = np.exp(self.log_std)


    def act(self, state, agent_i):
        # Use the network to predict the next action to take, using the model
        if agent_i==1:
          pred = self.Actor_env1.predict(state)
        elif agent_i==2:
          pred = self.Actor_env2.predict(state)

        low, high = -1.0, 1.0 # -1 and 1 are boundaries of tanh
        action = pred + np.random.uniform(low, high, size=pred.shape) * self.std
        action = np.clip(action, low, high)
        
        logp_t = self.gaussian_likelihood(action, pred, self.log_std)

        return action, logp_t

    def gaussian_likelihood(self, action, pred, log_std):
        # https://github.com/hill-a/stable-baselines/blob/master/stable_baselines/sac/policies.py
        pre_sum = -0.5 * (((action-pred)/(np.exp(log_std)+1e-8))**2 + 2*log_std + np.log(2*np.pi)) 
        return np.sum(pre_sum, axis=1)

    def discount_rewards(self, reward):#gaes is better
        # Compute the gamma-discounted rewards over an episode
        # We apply the discount and normalize it to avoid big variability of rewards
        gamma = 0.99    # discount rate
        running_add = 0
        discounted_r = np.zeros_like(reward)
        for i in reversed(range(0,len(reward))):
            running_add = running_add * gamma + reward[i]
            discounted_r[i] = running_add

        discounted_r -= np.mean(discounted_r) # normalizing the result
        discounted_r /= (np.std(discounted_r) + 1e-8) # divide by standard deviation
        return discounted_r

    def get_gaes(self, rewards, dones, values, next_values, gamma = 0.99, lamda = 0.90, normalize=True):
        deltas = [r + gamma * (1 - d) * nv - v for r, d, nv, v in zip(rewards, dones, next_values, values)]
        deltas = np.stack(deltas)
        gaes = copy.deepcopy(deltas)
        for t in reversed(range(len(deltas) - 1)):
            gaes[t] = gaes[t] + (1 - dones[t]) * gamma * lamda * gaes[t + 1]

        target = gaes + values
        if normalize:
            gaes = (gaes - gaes.mean()) / (gaes.std() + 1e-8)
        return np.vstack(gaes), np.vstack(target)

    def replay(self, states, actions, rewards, dones, next_states, logp_ts, agent_i):
        # reshape memory to appropriate shape for training
        states = np.vstack(states)
        next_states = np.vstack(next_states)
        actions = np.vstack(actions)
        logp_ts = np.vstack(logp_ts)

        # Get Critic network predictions 
        if agent_i==1:
          values = self.Critic_env1.predict(states)
          next_values = self.Critic_env1.predict(next_states)
        elif agent_i==2:
          values = self.Critic_env2.predict(states)
          next_values = self.Critic_env2.predict(next_states)
          
        # Compute discounted rewards and advantages
        #discounted_r = self.discount_rewards(rewards)
        #advantages = np.vstack(discounted_r - values)
        advantages, target = self.get_gaes(rewards, dones, np.squeeze(values), np.squeeze(next_values))
        '''
        pylab.plot(adv,'.')
        pylab.plot(target,'-')
        ax=pylab.gca()
        ax.grid(True)
        pylab.subplots_adjust(left=0.05, right=0.98, top=0.96, bottom=0.06)
        pylab.show()
        if str(episode)[-2:] == "00": pylab.savefig(self.env_name+"_"+self.episode+".png")
        '''
        # stack everything to numpy array
        # pack all advantages, predictions and actions to y_true and when they are received
        # in custom loss function we unpack it
        y_true = np.hstack([advantages, actions, logp_ts])
        
        # training Actor and Critic networks
        if agent_i==1:
          a_loss_env1 = self.Actor_env1.Actor.fit(states, y_true, epochs=self.epochs, verbose=0, shuffle=self.shuffle)
          c_loss_env1 = self.Critic_env1.Critic.fit([states, values], target, epochs=self.epochs, verbose=0, shuffle=self.shuffle)
          # calculate loss parameters (should be done in loss, but couldn't find working way how to do that with disabled eager execution)
          pred = self.Actor_env1.predict(states)
          log_std = -0.5 * np.ones(self.action_size, dtype=np.float32)
          logp = self.gaussian_likelihood(actions, pred, log_std)
          approx_kl = np.mean(logp_ts - logp)
          approx_ent = np.mean(-logp)
          self.writer.add_scalar('Data/actor_loss_per_replay_env1', np.sum(a_loss_env1.history['loss']), self.replay_count_env1)
          self.writer.add_scalar('Data/critic_loss_per_replay_env1', np.sum(c_loss_env1.history['loss']), self.replay_count_env1)
          self.writer.add_scalar('Data/approx_kl_per_replay_env1', approx_kl, self.replay_count_env1)
          self.writer.add_scalar('Data/approx_ent_per_replay_env1', approx_ent, self.replay_count_env1)
          self.replay_count_env1 += 1
        elif agent_i==2:
          a_loss_env2 = self.Actor_env2.Actor.fit(states, y_true, epochs=self.epochs, verbose=0, shuffle=self.shuffle)
          c_loss_env2 = self.Critic_env2.Critic.fit([states, values], target, epochs=self.epochs, verbose=0, shuffle=self.shuffle)
          # calculate loss parameters (should be done in loss, but couldn't find working way how to do that with disabled eager execution)
          pred = self.Actor_env2.predict(states)
          log_std = -0.5 * np.ones(self.action_size, dtype=np.float32)
          logp = self.gaussian_likelihood(actions, pred, log_std)
          approx_kl = np.mean(logp_ts - logp)
          approx_ent = np.mean(-logp)
          self.writer.add_scalar('Data/actor_loss_per_replay_env2', np.sum(a_loss_env2.history['loss']), self.replay_count_env2)
          self.writer.add_scalar('Data/critic_loss_per_replay_env2', np.sum(c_loss_env2.history['loss']), self.replay_count_env2)
          self.writer.add_scalar('Data/approx_kl_per_replay_env2', approx_kl, self.replay_count_env2)
          self.writer.add_scalar('Data/approx_ent_per_replay_env2', approx_ent, self.replay_count_env2)
          self.replay_count_env2 += 1
        
 
    def load(self):
        self.Actor_env1.Actor.load_weights(self.Actor_name_env1)
        self.Actor_env2.Actor.load_weights(self.Actor_name_env2)
        self.Critic_env1.Critic.load_weights(self.Critic_name_env1)
        self.Critic_env2.Critic.load_weights(self.Critic_name_env2)

    def save_env1(self):
        self.Actor_env1.Actor.save_weights(self.Actor_name_env1)
        self.Critic_env1.Critic.save_weights(self.Critic_name_env1)
    def save_env2(self):
        self.Actor_env2.Actor.save_weights(self.Actor_name_env2)
        self.Critic_env2.Critic.save_weights(self.Critic_name_env2)
    def save(self):  
        self.save_env1() 
        self.save_env2() 

    pylab.figure(figsize=(18, 9))
    pylab.subplots_adjust(left=0.05, right=0.98, top=0.96, bottom=0.06)
    def PlotModel_env1(self, score, episode, save=True):
        self.scores_.append(score)
        self.episodes_.append(episode)
        self.average_.append(sum(self.scores_[-50:]) / len(self.scores_[-50:]))
        if str(episode)[-2:] == "00":# much faster than episode % 100
            pylab.plot(self.episodes_, self.scores_, 'b')
            pylab.plot(self.episodes_, self.average_, 'r')
            pylab.ylabel('Score', fontsize=18)
            pylab.xlabel('Steps', fontsize=18)
            try:
                pylab.grid(True)
                pylab.savefig(self.env_name+"_env1.png")
            except OSError:
                pass
        # saving best models
        if self.average_[-1] >= self.max_average_env1 and save:
            self.max_average_env1 = self.average_[-1]
            self.save() #self.save_env1()
            SAVING = "SAVING"
            # decreaate learning rate every saved model
            #self.lr *= 0.99
            #K.set_value(self.Actor_env1.Actor.optimizer.learning_rate, self.lr)
            #K.set_value(self.Critic_env1.Critic.optimizer.learning_rate, self.lr)
        else:
            SAVING = ""

        return self.average_[-1], SAVING

    def PlotModel_env2(self, score, episode, save=True):
        self.scores_.append(score)
        self.episodes_.append(episode)
        self.average_.append(sum(self.scores_[-50:]) / len(self.scores_[-50:]))
        if str(episode)[-2:] == "00":# much faster than episode % 100
            pylab.plot(self.episodes_, self.scores_, 'b')
            pylab.plot(self.episodes_, self.average_, 'r')
            pylab.ylabel('Score', fontsize=18)
            pylab.xlabel('Steps', fontsize=18)
            try:
                pylab.grid(True)
                pylab.savefig(self.env_name+"_env2.png")
            except OSError:
                pass
        # saving best models
        if self.average_[-1] >= self.max_average_env2 and save:
            self.max_average_env2 = self.average_[-1]
            self.save() #self.save_env2()
            SAVING = "SAVING"
            # decreaate learning rate every saved model
            #self.lr *= 0.99
            #K.set_value(self.Actor_env1.Actor.optimizer.learning_rate, self.lr)
            #K.set_value(self.Critic_env1.Critic.optimizer.learning_rate, self.lr)
        else:
            SAVING = ""

        return self.average_[-1], SAVING
  
    def run_batch(self):
        state = self.env.reset() #total observe   [array([ 2.7, ....]),array([ 2.7, ....])]  list
        state_env1 = state[0] # env observation agent 1 / with communication [ 2.7, ....]   array
        state_env2 = state[1]
        state_env1 = np.reshape(state_env1, [1, self.state_size[0]])
        state_env2 = np.reshape(state_env2, [1, self.state_size[0]])
        done, score_env1, score_env2, SAVING_env1, SAVING_env2 = False, 0, 0, '', ''  #!!!!!
        while True:
            # Instantiate or reset games memory
            states_env1, next_states_env1, actions_env1, rewards_env1, logp_ts_env1, dones = [], [], [], [], [], []  #!!!!!
            states_env2, next_states_env2, actions_env2, rewards_env2, logp_ts_env2 = [], [], [], [], []  #!!!!!
            for t in range(self.Training_batch):
                self.env.render()
                # Actor picks an action
                action_env1, logp_t_env1 = self.act(state_env1, 1)
                action_env2, logp_t_env2 = self.act(state_env2, 2)
                # Retrieve new state, reward, and whether the state is terminal
                action = np.array([action_env1[0], action_env2[0]])
                next_state, reward, done, _ = self.env.step(action)
                # Memorize (state, next_states, action, reward, done, logp_ts) for training
                next_state_env1 = next_state[0] 
                next_state_env2 = next_state[1]
                next_state_env1 = np.reshape(next_state_env1, [1, self.state_size[0]])
                next_state_env2 = np.reshape(next_state_env2, [1, self.state_size[0]])
                reward_env1 = reward[0]
                reward_env2 = reward[1]
                states_env1.append(state_env1)
                states_env2.append(state_env2)
                next_states_env1.append(next_state_env1)
                next_states_env2.append(next_state_env2)
                actions_env1.append(action_env1)
                actions_env2.append(action_env2)
                rewards_env1.append(reward_env1)
                rewards_env2.append(reward_env2)
                dones.append(done)
                logp_ts_env1.append(logp_t_env1[0])
                logp_ts_env2.append(logp_t_env2[0])
                # Update current state shape
                state_env1 = next_state_env1
                state_env2 = next_state_env2
                score_env1 += reward_env1
                score_env2 += reward_env2
                if done:
                    self.episode += 1
                    average_env1, SAVING_env1 = self.PlotModel_env1(score_env1, self.episode)
                    average_env2, SAVING_env2 = self.PlotModel_env2(score_env2, self.episode)
                    print("Env1:: episode: {}/{}, score: {}, average: {:.2f} {}".format(self.episode, self.EPISODES, score_env1, average_env1, SAVING_env1))
                    print("Env2:: episode: {}/{}, score: {}, average: {:.2f} {}".format(self.episode, self.EPISODES, score_env2, average_env2, SAVING_env2))
                    self.writer.add_scalar(f'Workers:{1}/score_per_episode_env1', score_env1, self.episode)
                    self.writer.add_scalar(f'Workers:{1}/score_per_episode_env2', score_env2, self.episode)
                    self.writer.add_scalar(f'Workers:{1}/learning_rate', self.lr, self.episode)
                    self.writer.add_scalar(f'Workers:{1}/average_score_env1',  average_env1, self.episode)
                    self.writer.add_scalar(f'Workers:{1}/average_score_env2',  average_env2, self.episode)
                    
                    state, done, score_env1, score_env2, SAVING_env1, SAVING_env2 = self.env.reset(), False, 0, 0, '', ''
                    state_env1 = state[0]
                    state_env2 = state[1]
                    state_env1 = np.reshape(state_env1, [1, self.state_size[0]])
                    state_env2 = np.reshape(state_env2, [1, self.state_size[0]])

            self.replay(states_env1, actions_env1, rewards_env1, dones, next_states_env1, logp_ts_env1, 1)
            self.replay(states_env2, actions_env2, rewards_env2, dones, next_states_env2, logp_ts_env2, 2)
            if self.episode >= self.EPISODES:
                break

        self.env.close()


    # def run_multiprocesses(self, num_worker = 4):
    #     works, parent_conns, child_conns = [], [], []
    #     for idx in range(num_worker):
    #         parent_conn, child_conn = Pipe()
    #         work = Environment(idx, child_conn, self.env_name, self.state_size[0], self.action_size, True)
    #         work.start()
    #         works.append(work)
    #         parent_conns.append(parent_conn)
    #         child_conns.append(child_conn)

    #     states =        [[] for _ in range(num_worker)]
    #     next_states =   [[] for _ in range(num_worker)]
    #     actions =       [[] for _ in range(num_worker)]
    #     rewards =       [[] for _ in range(num_worker)]
    #     dones =         [[] for _ in range(num_worker)]
    #     logp_ts =       [[] for _ in range(num_worker)]
    #     score =         [0 for _ in range(num_worker)]

    #     state = [0 for _ in range(num_worker)]
    #     for worker_id, parent_conn in enumerate(parent_conns):
    #         state[worker_id] = parent_conn.recv()

    #     while self.episode < self.EPISODES:
    #         # get batch of action's and log_pi's
    #         action, logp_pi = self.act(np.reshape(state, [num_worker, self.state_size[0]]))
            
    #         for worker_id, parent_conn in enumerate(parent_conns):
    #             parent_conn.send(action[worker_id])
    #             actions[worker_id].append(action[worker_id])
    #             logp_ts[worker_id].append(logp_pi[worker_id])

    #         for worker_id, parent_conn in enumerate(parent_conns):
    #             next_state, reward, done, _ = parent_conn.recv()

    #             states[worker_id].append(state[worker_id])
    #             next_states[worker_id].append(next_state)
    #             rewards[worker_id].append(reward)
    #             dones[worker_id].append(done)
    #             state[worker_id] = next_state
    #             score[worker_id] += reward

    #             if done:
    #                 average, SAVING = self.PlotModel(score[worker_id], self.episode)
    #                 print("episode: {}/{}, worker: {}, score: {}, average: {:.2f} {}".format(self.episode, self.EPISODES, worker_id, score[worker_id], average, SAVING))
    #                 self.writer.add_scalar(f'Workers:{num_worker}/score_per_episode', score[worker_id], self.episode)
    #                 self.writer.add_scalar(f'Workers:{num_worker}/learning_rate', self.lr, self.episode)
    #                 self.writer.add_scalar(f'Workers:{num_worker}/average_score',  average, self.episode)
    #                 score[worker_id] = 0
    #                 if(self.episode < self.EPISODES):
    #                     self.episode += 1
                        
                        
    #         for worker_id in range(num_worker):
    #             if len(states[worker_id]) >= self.Training_batch:
    #                 self.replay(states[worker_id], actions[worker_id], rewards[worker_id], dones[worker_id], next_states[worker_id], logp_ts[worker_id])

    #                 states[worker_id] = []
    #                 next_states[worker_id] = []
    #                 actions[worker_id] = []
    #                 rewards[worker_id] = []
    #                 dones[worker_id] = []
    #                 logp_ts[worker_id] = []

    #     # terminating processes after a while loop
    #     works.append(work)
    #     for work in works:
    #         work.terminate()
    #         print('TERMINATED:', work)
    #         work.join()

    def test(self, test_episodes = 100):#evaluate
        self.load()
        video_recorder = None
        video_recorder = VideoRecorder(self.env,'./test.mp4', enabled=True)
        for e in range(test_episodes):
            state = self.env.reset()
            state_env1 = state[0]
            state_env2 = state[1]
            state_env1 = np.reshape(state_env1, [1, self.state_size[0]])
            state_env2 = np.reshape(state_env2, [1, self.state_size[0]])
            done = False
            score = 0
            while not done:
                self.env.render()
                action_env1 = self.Actor_env1.predict(state_env1)[0]
                action_env2 = self.Actor_env2.predict(state_env2)[0]
                action = np.array([action_env1[0], action_env2[0]])
                state, reward, done, _ = self.env.step(action)
                state_env1 = state[0]
                state_env2 = state[1]
                state_env1 = np.reshape(state_env1, [1, self.state_size[0]])
                state_env2 = np.reshape(state_env2, [1, self.state_size[0]])
                reward_env1 = reward[0]
                reward_env2 = reward[1]
                score_env1 += reward_env1
                score_env2 += reward_env2
                if done:
                    average_env1, SAVING_env1 = self.PlotModel_env1(score_env1, e, save=False)
                    average_env2, SAVING_env2 = self.PlotModel_env2(score_env2, e, save=False)
                    print("Env1:: episode: {}/{}, score: {}, average: {:.2f} {}".format(e+1, test_episodes, score_env1, average_env1))
                    print("Env2:: episode: {}/{}, score: {}, average: {:.2f} {}".format(e+1, test_episodes, score_env2, average_env2))
                    break
        self.env.close()
        print('Saved video.')
        video_recorder.close()
        video_recorder.enabled = False
        self.env.close()
        show_video('test')

if __name__ == "__main__":
    # newest gym fixed bugs in 'BipedalWalker-v2' and now it's called 'BipedalWalker-v3'
    env_name = 'MultiWalkerEnv'
    agent = PPOAgent(env_name)
    #agent.run_batch() # train as PPO
    #agent.run_multiprocesses(num_worker = 16)  # train PPO multiprocessed (fastest)
    #agent.test()

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


<Figure size 1296x648 with 0 Axes>

In [ ]:
agent.run_batch() # train as PPO
#!rm -rf ./runs
#agent.test(5)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Streaming output truncated to the last 5000 lines.
Env1:: episode: 746/200000, score: -48.093869581469335, average: -48.48 
Env2:: episode: 746/200000, score: -48.093869581469335, average: -48.39 
Env1:: episode: 747/200000, score: -46.67478310371128, average: -48.47 
Env2:: episode: 747/200000, score: -46.67478310371128, average: -48.54 
Env1:: episode: 748/200000, score: -50.11678948276676, average: -48.58 
Env2:: episode: 748/200000, score: -50.11678948276676, average: -48.61 
Env1:: episode: 749/200000, score: -58.4175051919495, average: -48.89 
Env2:: episode: 749/200000, score: -58.4175051919495, average: -49.17 
Env1:: episode: 750/200000, score: -49.43380967795383, average: -49.17 
Env2:: episode: 750/200000, score: -49.43380967795383, average: -49.17 
Env1:: episode: 751/200000, score: -45.998690203569524, average: -49.08 
Env2:: episode: 751/200000, score: -45.998690203569524, average: -48.99 
Env1:: episode: 752/200000, score: -51.70778605380717, average: -48.98 
Env2:: epis